In [ ]:
import optuna 
import tensorflow as tf
from tensorflow.keras import layers,models,regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

/home/alok_kumar/miniconda3/envs/cv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/data/train',target_size=(28,28),batch_size=32,class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    'data/data/val',target_size=(28,28),batch_size=32,class_mode='sparse'
)

def create_model(trial):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(28,28,3)))

    n_conv_layer = trial.suggest_int('n_conv_layers',2,4)
    kernel_reg = trial.suggest_loguniform('kernel_reg',1e-5,1e-2)
    conv_dropout = trial.suggest_float('conv_dropout',0.1,0.5)

    activation_type = trial.suggest_categorical('activation',['relu','elu','leakyrelu'])

    if activation_type == 'leakyrelu':
        alpha = trial.suggest_float('leaky_alpha',0.05,0.3)
    
    for i in range(n_conv_layer):
        filters = trial.suggest_categorical(f"filter_layer_{i}",[32, 64, 128, 256])
        model.add(layers.Conv2D(filters,(3,3),padding='same',
                                kernel_regularizer= regularizers.l2(kernel_reg)))
        model.add(layers.BatchNormalization())

        if activation_type == 'leakyrelu':
            model.add(layers.LeakyReLU(alpha = alpha))
        else:
            model.add(layers.Activation(activation_type))
        
        model.add(layers.MaxPool2D((2,2)))
        model.add(layers.Dropout(conv_dropout))
    
    model.add(layers.GlobalAveragePooling2D())

    n_dense_layer = trial.suggest_int('n_dense_layers',1,4)
    dense_dropout = trial.suggest_float('dense_dropout',0.2,0.6)

    for j in range(n_dense_layer):
        units = trial.suggest_categorical(f"dense_units_{j}",[32,64, 128, 256])
        model.add(layers.Dense(units,kernel_regularizer=regularizers.l2(kernel_reg)))
        model.add(layers.BatchNormalization())

        if activation_type == 'leakyrelu':
            model.add(layers.LeakyReLU(alpha = alpha))
        else:
            model.add(layers.Activation(activation_type))

        model.add(layers.Dropout(dense_dropout))

    model.add(layers.Dense(36,activation='softmax'))

    model.compile(
        optimizer = 'adam',
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )

    return model


def objective(trial):
    model = create_model(trial)

    es = EarlyStopping(monitor='val_accuracy',patience=15,restore_best_weights=True,verbose=0)

    history = model.fit(
        train_generator,
        validation_data= test_generator,
        epochs = 50,
        callbacks = [es],
        verbose = 1
    )

    val_acc = max(history.history['val_accuracy'])
    return val_acc

study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=20)

Found 864 images belonging to 36 classes.
Found 216 images belonging to 36 classes.


[I 2025-11-09 12:11:32,395] A new study created in memory with name: no-name-b6abbdf1-4447-4c49-a6c7-5e5b05b3fe96
/tmp/ipykernel_1436/268965132.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  kernel_reg = trial.suggest_loguniform('kernel_reg',1e-5,1e-2)


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - accuracy: 0.0663 - loss: 6.4058 - val_accuracy: 0.0741 - val_loss: 5.4352
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2040 - loss: 5.0387 - val_accuracy: 0.0556 - val_loss: 6.1377
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4102 - loss: 4.1951 - val_accuracy: 0.0787 - val_loss: 7.3314
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4846 - loss: 3.7889 - val_accuracy: 0.0509 - val_loss: 8.3928
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6126 - loss: 3.3377 - val_accuracy: 0.0648 - val_loss: 9.1087
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6211 - loss: 3.1628 - val_accuracy: 0.0880 - val_loss: 9.0702
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6617 - loss: 2.9622 - val_accuracy: 0.0694 - val_loss: 9.5385
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7647 - loss: 2.7096 - val_accuracy: 0.0741 - 

[I 2025-11-09 12:12:02,650] Trial 0 finished with value: 0.9953703880310059 and parameters: {'n_conv_layers': 3, 'kernel_reg': 0.0035651979050816763, 'conv_dropout': 0.43818790696863485, 'activation': 'elu', 'filter_layer_0': 32, 'filter_layer_1': 256, 'filter_layer_2': 256, 'n_dense_layers': 1, 'dense_dropout': 0.59956746327149, 'dense_units_0': 256}. Best is trial 0 with value: 0.9953703880310059.


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 105ms/step - accuracy: 0.0296 - loss: 4.6199 - val_accuracy: 0.0278 - val_loss: 4.0355
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0326 - loss: 4.4972 - val_accuracy: 0.0278 - val_loss: 4.0474
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0377 - loss: 4.2498 - val_accuracy: 0.0278 - val_loss: 4.0531
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0416 - loss: 4.2815 - val_accuracy: 0.0370 - val_loss: 4.0483
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0588 - loss: 4.0870 - val_accuracy: 0.0463 - val_loss: 4.0133
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0674 - loss: 3.9481 - val_accuracy: 0.0741 - val_loss: 3.9760
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0919 - loss: 3.9068 - val_accuracy: 0.0556 - val_loss: 3.9399
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0908 - loss: 3.7260 - val_accuracy: 0.0556 -

...

[I 2025-11-09 12:19:46,203] Trial 17 finished with value: 0.9629629850387573 and parameters: {'n_conv_layers': 2, 'kernel_reg': 0.00015855253935289265, 'conv_dropout': 0.2947710855781184, 'activation': 'relu', 'filter_layer_0': 256, 'filter_layer_1': 64, 'n_dense_layers': 1, 'dense_dropout': 0.24863166269049725, 'dense_units_0': 128}. Best is trial 2 with value: 1.0.


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 103ms/step - accuracy: 0.0425 - loss: 4.2897 - val_accuracy: 0.0417 - val_loss: 3.5910
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0415 - loss: 3.9435 - val_accuracy: 0.0370 - val_loss: 3.5815
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0458 - loss: 3.8433 - val_accuracy: 0.0648 - val_loss: 3.5812
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0666 - loss: 3.7177 - val_accuracy: 0.0694 - val_loss: 3.5629
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0752 - loss: 3.5666 - val_accuracy: 0.0509 - val_loss: 3.5610
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1233 - loss: 3.2901 - val_accuracy: 0.0278 - val_loss: 3.5724
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ -0s -18970us/step - accuracy: 0.1231 - loss: 3.1752 - val_accuracy: 0.0370 - val_loss: 3.5545
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1872 - loss: 2.9222 - val_accuracy: 0.0

[I 2025-11-09 12:20:20,511] Trial 18 finished with value: 0.9953703880310059 and parameters: {'n_conv_layers': 4, 'kernel_reg': 2.5908276260933143e-05, 'conv_dropout': 0.3772062223480077, 'activation': 'relu', 'filter_layer_0': 256, 'filter_layer_1': 32, 'filter_layer_2': 256, 'filter_layer_3': 64, 'n_dense_layers': 2, 'dense_dropout': 0.45145504699971367, 'dense_units_0': 64, 'dense_units_1': 128}. Best is trial 2 with value: 1.0.


Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - accuracy: 0.0529 - loss: 4.2216 - val_accuracy: 0.0694 - val_loss: 3.4296
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1364 - loss: 3.3399 - val_accuracy: 0.0463 - val_loss: 4.9961
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2449 - loss: 2.5883 - val_accuracy: 0.0231 - val_loss: 7.2529
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3413 - loss: 2.0742 - val_accuracy: 0.0417 - val_loss: 8.1875
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4905 - loss: 1.6460 - val_accuracy: 0.0694 - val_loss: 8.9036
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5504 - loss: 1.4089 - val_accuracy: 0.0278 - val_loss: 10.1268
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6108 - loss: 1.1984 - val_accuracy: 0.0602 - val_loss: 10.1826
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6208 - loss: 1.1183 - val_accuracy: 0.0556 

[I 2025-11-09 12:20:49,248] Trial 19 finished with value: 0.9953703880310059 and parameters: {'n_conv_layers': 3, 'kernel_reg': 2.398421731163282e-05, 'conv_dropout': 0.45145241129132246, 'activation': 'elu', 'filter_layer_0': 128, 'filter_layer_1': 128, 'filter_layer_2': 256, 'n_dense_layers': 3, 'dense_dropout': 0.3435677778771776, 'dense_units_0': 256, 'dense_units_1': 128, 'dense_units_2': 256}. Best is trial 2 with value: 1.0.


In [11]:
study.best_trial

FrozenTrial(number=2, state=TrialState.COMPLETE, values=[1.0], datetime_start=datetime.datetime(2025, 11, 9, 12, 12, 38, 29623), datetime_complete=datetime.datetime(2025, 11, 9, 12, 13, 11, 442150), params={'n_conv_layers': 3, 'kernel_reg': 1.2939633011200563e-05, 'conv_dropout': 0.2638297627401004, 'activation': 'relu', 'filter_layer_0': 256, 'filter_layer_1': 128, 'filter_layer_2': 256, 'n_dense_layers': 2, 'dense_dropout': 0.20334071801248632, 'dense_units_0': 64, 'dense_units_1': 128}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_conv_layers': IntDistribution(high=4, log=False, low=2, step=1), 'kernel_reg': FloatDistribution(high=0.01, log=True, low=1e-05, step=None), 'conv_dropout': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'activation': CategoricalDistribution(choices=('relu', 'elu', 'leakyrelu')), 'filter_layer_0': CategoricalDistribution(choices=(32, 64, 128, 256)), 'filter_layer_1': CategoricalDistribution(choices=(32, 64, 128, 25

In [12]:
study.best_params

{'n_conv_layers': 3,
 'kernel_reg': 1.2939633011200563e-05,
 'conv_dropout': 0.2638297627401004,
 'activation': 'relu',
 'filter_layer_0': 256,
 'filter_layer_1': 128,
 'filter_layer_2': 256,
 'n_dense_layers': 2,
 'dense_dropout': 0.20334071801248632,
 'dense_units_0': 64,
 'dense_units_1': 128}

In [13]:
study.best_value

1.0